In [7]:
# This makes an atlas out of a set of identically sized images. (c) Beherith mysterme@gmail.com

from PIL import Image

def create_image_atlas(image_list, atlas_size):
    # Calculate the number of rows and columns based on the total number of images
    num_images = len(image_list)
    num_rows = 32
    num_cols = 32
    positions = {}

    # Calculate the size of each image in the atlas
    image_width = atlas_size[0] // num_cols
    image_height = atlas_size[1] // num_rows

    # Create a new blank atlas image
    atlas = Image.new("RGBA", atlas_size)

    # Iterate over the image list and paste each image onto the atlas
    for i, image_path in enumerate(image_list):
        # Open the image using PIL
        image = Image.open(image_path)
        # Resize the image to fit the calculated size
        image = image.resize((image_width, image_height), Image.LANCZOS )

        # Calculate the position to paste the image onto the atlas
        x = (i % num_cols) * image_width
        y = (i // num_cols) * image_height

        # Paste the resized image onto the atlas
        atlas.paste(image, (x, y))

        #png origin is top left vs dds bottom left!
        halftexelX = 0.5 / atlas_size[0]
        halftexelY = 0.5 / atlas_size[1]
        ax = 1.0 - (x + image_width) / atlas_size[0] + halftexelX
        aX = 1.0 - (x) / atlas_size[0] - halftexelX
        ay =  (y) / atlas_size[1] + halftexelY
        aY = (y+image_height) / atlas_size[1] - halftexelY
        aw = (image_width) / atlas_size[0]
        ah = (image_height) / atlas_size[1]
        positions[image_path] = f"{{ {ax},{aX},{ay},{aY},{aw},{ah} }} ,  --xXyYwh"
        #print(positions)
    return atlas, positions


# Example usage
#image_paths = ["image1.jpg", "image2.jpg", "image3.jpg", "image4.jpg"]
#atlas_size = (800, 600)

#atlas_image = create_image_atlas(image_paths, atlas_size)
#atlas_image.show()
#atlas_image.save("atlas.jpg")

In [8]:
import os
image_paths = [filename for filename in os.listdir('.') if filename.lower().endswith('.png')]
image_paths += ['inverted/' + filename for filename in os.listdir('./inverted/') if filename.lower().endswith('.png')]
print (len(image_paths), image_paths[0:10])
#each img is targeted to be 32x32, 700 total, 

716 ['aa.png', 'aa_flak_1.4.png', 'aa_longrange_1.8.png', 'aa_t15.png', 'air.png', 'air_assistdrone.png', 'air_battleship.png', 'air_bladew.png', 'air_bomber.png', 'air_bomber_juno.png']


In [9]:
atlas_image, positions = create_image_atlas(image_paths, (1024,1024))
#atlas_image.show()
atlas_image.save("icon_atlas.png")

luafile = open("icon_atlas.lua",'w')
luafile.write('local atlas = {\n')
for fname in positions.keys():
    luafile.write('\t[icons/%s] = %s \n'%(fname, positions[fname]))
luafile.write('\n}\nreturn atlas\n')
luafile.close()

C:\Users\Peti\AppData\Local\Temp/ipykernel_33628/120163981.py:24: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  image = image.resize((image_width, image_height), Image.LANCZOS )


In [10]:
#finally convert to dds with
# \springrts_smf_compiler\NVTT_DragAndDropConvertToDDSTools\nvtt_export.exe" --output "icon_atlas.dds" --mip-filter 0 --quality 3 --format bc3 "icon_atlas.png"